In [28]:
import numpy as np
import cv2 as cv
import glob
import os

In [29]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

In [30]:
image_files = glob.glob("images_2024-11-23_18-13_rae_right_image_raw/*.jpg")

In [31]:
len(image_files)

254

In [32]:
obj_point = np.zeros((8*5, 3), np.float32)
grid = np.mgrid[0:8,0:5]
obj_point[:,:2] = grid.T.reshape(-1,2)


In [41]:
chessboard_size = (8,5)
square_size = 2.6

In [42]:
obj_point = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
obj_point[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
obj_point = obj_point * square_size 

In [43]:
obj_points = []
img_points = []

In [45]:
for filename in image_files:
    img = cv.imread(filename)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
 
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (8,5), None)
 
    # If found, add object points, image points (after refining them)
    if ret == True:
        obj_points.append(obj_point)
 
        #corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), None)
        img_points.append(corners)
 
        # Draw and display the corners
        cv.drawChessboardCorners(img, (8,5), corners, ret)
        cv.imshow('img', img)
        # cv.waitKey(500)
cv.destroyAllWindows()

In [46]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(obj_points, img_points, gray.shape[::-1], None, None)

In [47]:
np.savetxt(fname="dist_new",X=dist)
np.savetxt(fname="mtx_new",X=mtx)

In [66]:
from PIL import Image
for i, filename in enumerate(image_files):
    img = cv.imread(filename)
    # print(filename, img)
    h,  w = img.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
    dst = cv.undistort(img, mtx, dist, None, newcameramtx)
 
    # crop the image
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    img = cv.cvtColor(dst, cv.COLOR_BGR2RGB)
    img = Image.fromarray(img)
    print(img.size, end='\r')
    img.save(f'/Users/lisann/Desktop/2024-2025/VAR/RAE/var_extensions/curling_pipeline/notebooks/calibresult/{i}.png')
    

In [65]:
newcameramtx, mtx

(array([[572.46246338,   0.        , 638.74817042],
        [  0.        , 575.39581299, 410.9380638 ],
        [  0.        ,   0.        ,   1.        ]]),
 array([[572.91005966,   0.        , 639.24762227],
        [  0.        , 576.11595282, 411.45237124],
        [  0.        ,   0.        ,   1.        ]]))